In [1]:
import sys
sys.path.append('..')

In [2]:
from src.ARKAutoML import ARKAutoML
from sklearn.model_selection import train_test_split
from config import BaseConfig
import pandas as pd

In [3]:
df = pd.read_csv(BaseConfig.FILE_NAME)

In [4]:
train, valid = train_test_split(df)

In [5]:
aml = ARKAutoML(train,config=BaseConfig,
                   eval_metric='accuracy', 
                   n_trials=20, model_algos=['lgb'],
                   )

2021-09-24 07:39:28.210 | INFO     | src.ARKAutoML.ARKAutoML:add_project_config_to_logs:41 - 
{'BASE_INPUT_PATH': '../input/',
 'BASE_OUTPUT_PATH': '../output/',
 'DATA_PATH': '../input/../input/train.csv',
 'FILE_NAME': '../input/train.csv',
 'FOLD_METHOD': 'StratifiedKFold',
 'PROBLEM_TYPE': 'classification',
 'PROJECT_NAME': 'MySampleProject',
 'SUB_PROJECT_NAME': 'Sample_1',
 'TARGET_COL': 'customer_category',
 'cat_cols': ['customer_active_segment', 'X1'],
 'cont_cols': ['customer_visit_score',
               'customer_product_search_score',
               'customer_ctr_score',
               'customer_stay_score',
               'customer_frequency_score',
               'customer_product_variation_score',
               'customer_order_score',
               'customer_affinity_score']}
2021-09-24 07:39:28.216 | INFO     | src.ARKAutoML.create_folds:create_folds:26 - Fold 1 : 6442, 1611
2021-09-24 07:39:28.219 | INFO     | src.ARKAutoML.create_folds:create_folds:26 - Fold 2 : 644

In [6]:
aml.n_folds = 1

In [7]:
aml.create_optuna_optimization()

[I 2021-09-24 07:39:28,487] A new study created in memory with name: MySampleProject
C:\ProgramData\Anaconda3\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.1, 100] and q=1, but the range is not divisible by `q`. It will be replaced by [0.1, 99.1].
  warnings.warn(
2021-09-24 07:39:28.553 | INFO     | src.ARKAutoML.ARKAutoML:objective:77 - Trial No : 0, accuracy : 0.9826, Params : {'model_algo': 'lgb', 'num_leaves': 40, 'min_child_samples': 225, 'min_child_weight': 3, 'subsample': 0.7, 'colsample_bytree': 0.6, 'reg_alpha': 39.1, 'reg_lambda': 72.1}
                        Accuracy : 0.983  F1_score_0 : 0.99  F1_score_1 : 0.924  Precision_0 : 0.985  Precision_1 : 0.966  Recall_0 : 0.996  Recall_1 : 0.885  Roc_auc : 0.985
[I 2021-09-24 07:39:28,772] Trial 0 finished with value: 0.9826194909993793 and parameters: {'model_algo': 'lgb', 'num_leaves': 40, 'min_child_samples': 225, 'min_child_weight': 3, 'subsample': 0.7, 'colsample_bytree': 0.

In [8]:
cross_validated_metrics, final_metrics_by_cut_off   = aml.get_crossvalidated_results_for_best_model(folds=5)

2021-09-24 07:39:31.130 | INFO     | src.ARKAutoML.ARKAutoML:get_crossvalidated_results_for_best_model:131 - Best Model : lgb, Model Params : {'num_leaves': 10, 'min_child_samples': 288, 'min_child_weight': 1, 'subsample': 1.0, 'colsample_bytree': 1.0, 'reg_alpha': 56.1, 'reg_lambda': 64.1}
2021-09-24 07:39:31.158 | INFO     | src.ARKAutoML.ARKAutoML:get_crossvalidated_results_for_best_model:137 - Fold : 1, Train Shape : (6442, 10), Test Shape : (1611, 10)
2021-09-24 07:39:31.174 | INFO     | src.ARKAutoML.metrics:model_metrics:43 - Accuracy : 0.983  Precision_1 : 0.966  Recall_1 : 0.891  F1_score_1 : 0.927  Precision_0 : 0.985  Recall_0 : 0.996  F1_score_0 : 0.991  Roc_auc : 0.986
2021-09-24 07:39:31.177 | INFO     | src.ARKAutoML.metrics:model_metrics:44 - 
 [[1413    6]
 [  21  171]]
2021-09-24 07:39:31.184 | INFO     | src.ARKAutoML.metrics:model_metrics:45 - 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      1419
           1 

In [9]:
aml.train_best_model()

2021-09-24 07:39:31.780 | INFO     | src.ARKAutoML.ARKAutoML:train_best_model:188 - Training "lgb" Model with Best Params : {'num_leaves': 10, 'min_child_samples': 288, 'min_child_weight': 1, 'subsample': 1.0, 'colsample_bytree': 1.0, 'reg_alpha': 56.1, 'reg_lambda': 64.1}
2021-09-24 07:39:31.843 | INFO     | src.ARKAutoML.metrics:model_metrics:43 - Accuracy : 0.975  Precision_1 : 0.94  Recall_1 : 0.839  F1_score_1 : 0.887  Precision_0 : 0.979  Recall_0 : 0.993  F1_score_0 : 0.986  Roc_auc : 0.973
2021-09-24 07:39:31.850 | INFO     | src.ARKAutoML.metrics:model_metrics:44 - 
 [[7043   51]
 [ 154  805]]
2021-09-24 07:39:31.864 | INFO     | src.ARKAutoML.metrics:model_metrics:45 - 
               precision    recall  f1-score   support

           0       0.98      0.99      0.99      7094
           1       0.94      0.84      0.89       959

    accuracy                           0.97      8053
   macro avg       0.96      0.92      0.94      8053
weighted avg       0.97      0.97     

In [10]:
score, metrics_by_cut_off = aml.score_validation_data(valid)

2021-09-24 07:39:31.905 | INFO     | src.ARKAutoML.ARKAutoML:score_validation_data:217 - Scoring Validation Data : Validation Shape : (2685, 10)
2021-09-24 07:39:31.924 | INFO     | src.ARKAutoML.metrics:model_metrics:43 - Accuracy : 0.975  Precision_1 : 0.935  Recall_1 : 0.863  F1_score_1 : 0.898  Precision_0 : 0.981  Recall_0 : 0.991  F1_score_0 : 0.986  Roc_auc : 0.976
2021-09-24 07:39:31.928 | INFO     | src.ARKAutoML.metrics:model_metrics:44 - 
 [[2329   20]
 [  46  290]]
2021-09-24 07:39:31.935 | INFO     | src.ARKAutoML.metrics:model_metrics:45 - 
               precision    recall  f1-score   support

           0       0.98      0.99      0.99      2349
           1       0.94      0.86      0.90       336

    accuracy                           0.98      2685
   macro avg       0.96      0.93      0.94      2685
weighted avg       0.97      0.98      0.97      2685

2021-09-24 07:39:32.000 | INFO     | src.ARKAutoML.ARKAutoML:score_validation_data:220 - Metrics by different C

In [11]:
aml.save_best_model()

2021-09-24 07:39:32.042 | INFO     | src.ARKAutoML.ARKAutoML:save_best_model:203 - ..\output\MySampleProject\Sample_1\models\best_model_preprocessor.pkl
2021-09-24 07:39:32.043 | INFO     | src.ARKAutoML.ARKAutoML:save_best_model:204 - ..\output\MySampleProject\Sample_1\models\best_model.pkl


In [12]:
test_d = valid.iloc[0:1, :].drop(BaseConfig.TARGET_COL,axis=1)

In [13]:
aml.predict(test_d)

(array([0], dtype=int8), array([[0.97296419, 0.02703581]]))

### Load PreTrained Model

In [14]:
aml2 = ARKAutoML(config=BaseConfig, loading=True)

2021-09-24 07:39:32.173 | INFO     | src.ARKAutoML.ARKAutoML:add_project_config_to_logs:41 - 
{'BASE_INPUT_PATH': '../input/',
 'BASE_OUTPUT_PATH': '../output/',
 'DATA_PATH': '../input/../input/train.csv',
 'FILE_NAME': '../input/train.csv',
 'FOLD_METHOD': 'StratifiedKFold',
 'PROBLEM_TYPE': 'classification',
 'PROJECT_NAME': 'MySampleProject',
 'SUB_PROJECT_NAME': 'Sample_1',
 'TARGET_COL': 'customer_category',
 'cat_cols': ['customer_active_segment', 'X1'],
 'cont_cols': ['customer_visit_score',
               'customer_product_search_score',
               'customer_ctr_score',
               'customer_stay_score',
               'customer_frequency_score',
               'customer_product_variation_score',
               'customer_order_score',
               'customer_affinity_score']}


In [15]:
aml2.load_best_model()

In [16]:
test_d = valid.iloc[0:1, :].drop(BaseConfig.TARGET_COL,axis=1)
aml2.predict(test_d)

(array([0], dtype=int8), array([[0.97296419, 0.02703581]]))

### Only Train on entire Data and Test on Valid

This method is not recommended. But can be used in case of low resources availability.

In [17]:
aml = ARKAutoML(train,
                   eval_metric='accuracy', 
                   n_folds=False, n_trials = 1, 
                   model_algos=['rf'],
                   config=BaseConfig)

2021-09-24 07:39:32.283 | INFO     | src.ARKAutoML.ARKAutoML:add_project_config_to_logs:41 - 
{'BASE_INPUT_PATH': '../input/',
 'BASE_OUTPUT_PATH': '../output/',
 'DATA_PATH': '../input/../input/train.csv',
 'FILE_NAME': '../input/train.csv',
 'FOLD_METHOD': 'StratifiedKFold',
 'PROBLEM_TYPE': 'classification',
 'PROJECT_NAME': 'MySampleProject',
 'SUB_PROJECT_NAME': 'Sample_1',
 'TARGET_COL': 'customer_category',
 'cat_cols': ['customer_active_segment', 'X1'],
 'cont_cols': ['customer_visit_score',
               'customer_product_search_score',
               'customer_ctr_score',
               'customer_stay_score',
               'customer_frequency_score',
               'customer_product_variation_score',
               'customer_order_score',
               'customer_affinity_score']}


In [18]:
aml.create_only_one_train_and_one_valid(valid)

In [19]:
aml.create_optuna_optimization()

[I 2021-09-24 07:39:32,391] A new study created in memory with name: MySampleProject
2021-09-24 07:39:33.996 | INFO     | src.ARKAutoML.ARKAutoML:objective:77 - Trial No : 0, accuracy : 0.9739, Params : {'model_algo': 'rf', 'max_depth': 8, 'n_estimators': 768, 'min_samples_split': 30, 'min_samples_leaf': 46}
                        Accuracy : 0.974  F1_score_0 : 0.985  F1_score_1 : 0.892  Precision_0 : 0.98  Precision_1 : 0.926  Recall_0 : 0.99  Recall_1 : 0.86  Roc_auc : 0.977
[I 2021-09-24 07:39:34,044] Trial 0 finished with value: 0.9739292364990689 and parameters: {'model_algo': 'rf', 'max_depth': 8, 'n_estimators': 768, 'min_samples_split': 30, 'min_samples_leaf': 46}. Best is trial 0 with value: 0.9739292364990689.
